# English Subtitles Level Prediction

# Подготовка словаря

---

**Входные данные**

Электронная таблица. Названия вкладок соответствуют названиям уровней CEFR. На каждой вкладке столбец со словами соответствующего уровня. Слова взяты из файлов "American_Oxford_3000_by_CEFR_level.pdf" и "American_Oxford_5000_by_CEFR_level.pdf".

---

**Цель**

Объединить слова в csv-файл, содержащий две колонки: 'Word' и 'Level'.

---

**Задачи:**  

- если одинаковое написание слова встречается в разных уровнях, присвоить слову наибольший (трудный) уровень.
- сгенерировать тексты с использованием исключительно слов одного уровня (для создания baseline-модели).

<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Initial" data-toc-modified-id="Initial-0"><span class="toc-item-num">0&nbsp;&nbsp;</span>Initial</a></span><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Constants" data-toc-modified-id="Constants-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Constants</a></span></li><li><span><a href="#Settings" data-toc-modified-id="Settings-0.3"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>Settings</a></span></li></ul></li><li><span><a href="#Подготовка-словаря-&quot;American-Oxford-by-CEFR-level&quot;" data-toc-modified-id="Подготовка-словаря-&quot;American-Oxford-by-CEFR-level&quot;-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка словаря "American Oxford by CEFR level"</a></span><ul class="toc-item"><li><span><a href="#Read-data" data-toc-modified-id="Read-data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Read data</a></span></li><li><span><a href="#Удаление-дубликатов" data-toc-modified-id="Удаление-дубликатов-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Удаление дубликатов</a></span></li></ul></li><li><span><a href="#Save-словарь" data-toc-modified-id="Save-словарь-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Save словарь</a></span></li><li><span><a href="#Создание-искусственных-текстов" data-toc-modified-id="Создание-искусственных-текстов-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Создание искусственных текстов</a></span></li></ul></div>

---

## Initial

### Imports

In [106]:
import pandas as pd
import numpy as np
import os
import warnings

### Constants

In [107]:
PATH_LOCAL = 'datasets/'                               # local path to data
PATH_REMOTE = '/datasets/'                             # remote path to data

CR = '\n'                                              # new line

### Settings

In [121]:
# text styles
class f:
    BOLD = "\033[1m"
    ITALIC = "\033[3m"
    END = "\033[0m"

In [111]:
# Pandas defaults
pd.options.display.max_colwidth = 100
pd.options.display.max_rows = 500
pd.options.display.max_columns = 100
pd.options.display.float_format = '{:.3f}'.format
pd.options.display.colheader_justify = 'left'

In [112]:
# others
warnings.filterwarnings('ignore')

---

## Подготовка словаря "American Oxford by CEFR level"

### Read data

Чтение с каждого листа и объединение в общий датафрейм.

In [113]:
data = pd.DataFrame()

for level in ['A1','A2','B1','B2','B2+','C1']:
    df = pd.read_excel(f'{PATH_LOCAL}The_Oxford_by_CEFR_level/word_level.xlsx', sheet_name=level)
    df['Level'] = level
    data = pd.concat([data, df], ignore_index=True)

In [114]:
data.Level = data.Level.replace('B2+', 'B2')

> Часть слов уровня B2 изначально помечены во входном файле как B2+.  
> Возможно, это излишне, и стоит их пометить в файле сразу как B2.

### Удаление дубликатов

In [115]:
data[['word']].duplicated().sum()

351

In [116]:
# дубликаты сортируются по убыванию признака `level`
df = data.sort_values('Level', ascending=False).drop('Level', axis=1)

# после удаления дубликатов остается только запись с максимальным значением `level`
data = data[~df.duplicated()]

In [117]:
data[['word']].duplicated().sum()

0

In [118]:
data.Level.value_counts()

B2    1406
C1    1310
B1     759
A1     737
A2     731
Name: Level, dtype: int64

> Если одно и то же слово (пусть даже в разных смыслах) присутствует в разных уовнях, ему будет присвоен наибольший из этих уовней. Для обучающегося лучше встреченное слово будет простым, чем незнакомым.

> Другой вариант – не удалять дубликаты, если это соответствует логике дальнейшей обработки.

## Save словарь

In [119]:
data.to_csv(f'{PATH_LOCAL}EDA_word_level.csv', index=False)

## Создание искусственных текстов

In [120]:
N = 100
df = pd.DataFrame()

for level in list(data.Level.unique()):

    for i in range(N):
        
        generated_text = data[data.Level == level].sample(frac=1).word.str.cat(sep=' ')
        new_row = {'Movie':'generated', 'Level':level, 'Subtitles':generated_text}
        df = df.append(new_row, ignore_index=True)

# save
df.to_csv(f'{PATH_LOCAL}EDA_artficial_subs.csv', index=False)

> Сгенерированные тексты (а точнее, просто наборы слов) состоят исключительно из слов какого-либо одного уровня.  
> На базе этих текстов будет создана baseline-модель.